In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

from bs4 import BeautifulSoup
import pandas as pd

## My first attempt to simulate scrolling

In [4]:
url = 'https://www.olx.co.id/jakarta-selatan_g4000030/motor-bekas_c200'
driver = webdriver.Chrome()
driver.get(url)

for i in range(100):
    time.sleep(2)

    # Try to find and click the "Muat lainnya" button
    try:
        load_more_button = WebDriverWait(driver, 4).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'div._38O09 > button'))
        )
        load_more_button.click()
        time.sleep(2)  # Wait for the listings to load
    except Exception as e:
        print("No more 'Muat lainnya' button found or failed to click:", e)
        break  # Stop if no more 'Muat lainnya' button is found

    time.sleep(3)

    html_source = driver.page_source
    soup = BeautifulSoup(html_source, 'html.parser')

    # Find all new links to listings
    links = soup.find_all('a', attrs={'class':'_2cbZ2'})
    link_list = []

    for link in links:
        link_list.append('https://www.olx.co.id' + link.get('href'))

link_list

No more 'Muat lainnya' button found or failed to click: Message: element click intercepted: Element <button type="button" data-aut-id="btnLoadMore" class="rui-apowA rui-htytx rui-UGVY0">...</button> is not clickable at point (679, 78). Other element would receive the click: <div class="_37tZ0">...</div>
  (Session info: chrome=129.0.6668.60)
Stacktrace:
	GetHandleVerifier [0x00007FF7470FB645+29573]
	(No symbol) [0x00007FF747070470]
	(No symbol) [0x00007FF746F2B6EA]
	(No symbol) [0x00007FF746F874EE]
	(No symbol) [0x00007FF746F84F3C]
	(No symbol) [0x00007FF746F82408]
	(No symbol) [0x00007FF746F8161A]
	(No symbol) [0x00007FF746F736BE]
	(No symbol) [0x00007FF746FA72FA]
	(No symbol) [0x00007FF746F72FF6]
	(No symbol) [0x00007FF746FA7510]
	(No symbol) [0x00007FF746FC86BC]
	(No symbol) [0x00007FF746FA70A3]
	(No symbol) [0x00007FF746F712DF]
	(No symbol) [0x00007FF746F72441]
	GetHandleVerifier [0x00007FF74742C58D+3375821]
	GetHandleVerifier [0x00007FF747477987+3684039]
	GetHandleVerifier [0x0000

['https://www.olx.co.id/item/yukk-serbu-dp-400-lngsng-gas-beat-2024-cash-credt-iid-926170920',
 'https://www.olx.co.id/item/honda-vario-2016-iid-926419319',
 'https://www.olx.co.id/item/dijual-honda-scoopy-tahun-2024-km-2-rb-like-a-new-siap-gas-iid-926299488',
 'https://www.olx.co.id/item/yamaha-xmax-hitam-tahun-2021-jakarta-selatan-iid-926314316',
 'https://www.olx.co.id/item/mulus-yamaha-aerox-new-connected-pajak-panjang-surat-lengkap-iid-926514088',
 'https://www.olx.co.id/item/promo-khusus-cukup-dp-500k-bawa-pulang-yamaha-all-new-nmax-2023-iid-926174415',
 'https://www.olx.co.id/item/rx-king-se-2003-iid-924799167',
 'https://www.olx.co.id/item/ktm-1290-super-adventure-t-2016-iid-925792591',
 'https://www.olx.co.id/item/yamaha-nmax-2016-ban-battlax-pajak-2025-iid-925608692',
 'https://www.olx.co.id/item/kawasaki-z800-2014-full-modif-iid-924668883',
 'https://www.olx.co.id/item/vespa-sprint-150-s-abs-blue-audace-rare--iid-924610703',
 'https://www.olx.co.id/item/cafe-racer-gl-200-cc-

In [5]:
len(link_list)

169

In [ ]:
listings_list1 = []

for link in link_list:  
    driver.get(link)
    html_source = driver.page_source
    soup = BeautifulSoup(html_source, 'html.parser')

    # Extract details (as per your original code)
    try:
        listing_title = soup.find('h1', attrs={'data-aut-id':'itemTitle'}).text
    except:
        listing_title = 'no title description'

    try:
        price = soup.find('span', attrs={'data-aut-id':'itemPrice'}).text
    except:
        price = 'Rp 0'
    
    try:
        location = soup.find('span', attrs={'class':'_1RkZP'}).text
    except:
        location = 'unknown'

    try:
        manufacturer = soup.find('span', attrs={'data-aut-id':'value_make'}).text
    except:
        manufacturer = 'unknown'

    try:
        model = soup.find('span', attrs={'data-aut-id':'value_m_tipe'}).text
    except:
        model = 'unknown'

    try:
        year = soup.find('span', attrs={'data-aut-id':'value_m_year'}).text
    except:
        year = 0

    try:
        mileage = soup.find('span', attrs={'data-aut-id':'value_mileage'}).text
    except:
        mileage = None

    try:
        desc = soup.find('div', attrs={'data-aut-id':'itemDescriptionContent'}).text.lower()
        tax = None
        if 'pajak' in desc:
            tax = desc.split('pajak')[1].split()[0]
    except:
        tax = 'unknown'

    try:
        seller = soup.find('span', attrs={'data-aut-id':'value_m_seller_type'}).text
    except:
        seller = 'unknown'
    
    listing_dict = {'listing_title': listing_title,
                    'price': price,
                    'location': location,
                    'manufacturer': manufacturer,
                    'model': model,
                    'year': year,
                    'mileage': mileage,
                    'tax': tax,
                    'seller_type': seller}
    
    listings_list1.append(listing_dict)

df = pd.DataFrame(listings_list1)

## My Second Attempt without Simulating Scrolling

In [10]:
html_source = driver.page_source
soup = BeautifulSoup(html_source, 'html.parser')
links = soup.find_all('a',attrs={'class':'_2cbZ2'})

In [11]:
link = 'https://www.olx.co.id' + links[5].get('href')
link

'https://www.olx.co.id/item/promo-khusus-cukup-dp-500k-bawa-pulang-yamaha-all-new-nmax-2023-iid-926174415'

In [12]:
link_list = []

for link in links:
    link_list.append('https://www.olx.co.id' + link.get('href'))

In [13]:
len(link_list)

169

In [71]:
manufacturer = soup.find('span', attrs={'data-aut-id':'value_make'}).text
model = soup.find('span', attrs={'data-aut-id':'value_m_tipe'}).text
year = soup.find('span', attrs={'data-aut-id':'value_m_year'}).text
listing_desc = soup.find('h1', attrs={'data-aut-id':'itemTitle'}).text
price = soup.find('span', attrs={'data-aut-id':'itemPrice'}).text
location = soup.find('span', attrs={'class':'_1RkZP'}).text

In [14]:
listings_list1 = []

for link in link_list:  
    driver.get(link)
    html_source = driver.page_source
    soup = BeautifulSoup(html_source, 'html.parser')

    # Extract details (as per your original code)
    try:
        listing_title = soup.find('h1', attrs={'data-aut-id':'itemTitle'}).text
    except:
        listing_title = 'no title description'

    try:
        price = soup.find('span', attrs={'data-aut-id':'itemPrice'}).text
    except:
        price = 'Rp 0'
    
    try:
        location = soup.find('span', attrs={'class':'_1RkZP'}).text
    except:
        location = 'unknown'

    try:
        manufacturer = soup.find('span', attrs={'data-aut-id':'value_make'}).text
    except:
        manufacturer = 'unknown'

    try:
        model = soup.find('span', attrs={'data-aut-id':'value_m_tipe'}).text
    except:
        model = 'unknown'

    try:
        year = soup.find('span', attrs={'data-aut-id':'value_m_year'}).text
    except:
        year = 0

    try:
        mileage = soup.find('span', attrs={'data-aut-id':'value_mileage'}).text
    except:
        mileage = None

    try:
        desc = soup.find('div', attrs={'data-aut-id':'itemDescriptionContent'}).text.lower()
        tax = None
        if 'pajak' in desc:
            tax = desc.split('pajak')[1].split()[0]
    except:
        tax = 'unknown'

    try:
        seller = soup.find('span', attrs={'data-aut-id':'value_m_seller_type'}).text
    except:
        seller = 'unknown'
    
    listing_dict = {'listing_title': listing_title,
                    'price': price,
                    'location': location,
                    'manufacturer': manufacturer,
                    'model': model,
                    'year': year,
                    'mileage': mileage,
                    'tax': tax,
                    'seller_type': seller}
    
    listings_list1.append(listing_dict)

df = pd.DataFrame(listings_list1)
df

,listing_title,price,location,manufacturer,model,year,mileage,tax,seller_type
0,(yukk serbu DP 400 lngsng gas) beat 2024 cash ...,Rp 15.300.000,"Pancoran, Jakarta Selatan, Jakarta D.K.I.",Honda,Beat,2024,5.000-10.000 km,panjang4.,Diler
1,Honda vario 2016,Rp 12.700.000,"Pasar Minggu, Jakarta Selatan, Jakarta D.K.I.",Honda,Vario,2016,0-5.000 km,hidupmotor,Individu
2,Dijual honda scoopy tahun 2024 km 2 rb like a...,Rp 21.350.000,"Pasar Minggu, Jakarta Selatan, Jakarta D.K.I.",Honda,Scoopy,2024,0-5.000 km,juni,Diler
3,Yamaha Xmax Hitam Tahun 2021 Jakarta Selatan,Rp 55.000.000,"Cilandak, Jakarta Selatan, Jakarta D.K.I.",Yamaha,Xmax,2021,10.000-15.000 km,sampai,Individu
4,MULUS Yamaha Aerox New Connected Pajak Panjang...,Rp 21.500.000,"Pancoran, Jakarta Selatan, Jakarta D.K.I.",Yamaha,Aerox,2021,25.000-30.000 km,hidup,Individu
...,...,...,...,...,...,...,...,...,...
164,DIJUAL VESPA PRIMAVERA 150 cc 2022 BEIGE COLOR,Rp 49.000.000,"Tanjung Duren, Jakarta Barat, Jakarta D.K.I.",Piaggio,Vespa,2022,0-5.000 km,None,Individu
165,MV Agusta Brutale 800 Italia Edition (Rare) 2013,Rp 389.000.000,"Pasar Rebo, Jakarta Timur, Jakarta D.K.I.",Lain-lain,Lain-lain,2013,10.000-15.000 km,None,Individu
166,YAMAHA TMAX 530 2018,Rp 395.000.000,"Kebon Jeruk, Jakarta Barat, Jakarta D.K.I.",Yamaha,T-Max 500,2018,10.000-15.000 km,off,Individu
167,Yamaha Vixion Custom Touring,Rp 12.000.000,"Ciputat, Tangerang Selatan Kota, Banten",Yamaha,Vixion,2009,None,None,Individu


In [15]:
df.to_csv('olx_satu')

Both ways result in the same length of data